In [1]:
import gc
import pickle
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from matplotlib.backends.backend_pdf import PdfPages


from boltzmann_machine import BoltzmannMachine


from utils import (plot_sigmoid,
                   get_encprob_env_states, 
                   get_encprob_weight_mask, 
                   gen_free_state_dist_pdf, 
                   gen_clamped_state_dist_pdf, 
                   gen_learning_plots_pdf,
                   get_boltzmann_distribution,
                   get_probs_from_counts,
                   plot_series,
                   stringify_vec)

grp_size = 4
num_hnodes = 2
env_states = get_encprob_env_states(grp_size)
weight_mask = get_encprob_weight_mask(grp_size, num_hnodes) # two vis grps should not interact with each other and hidden units should not interact with each other

In [2]:
def get_encNetwork_scores(bm):
    uniqs_len_list = []
    for clamped_debug_list in (bm.clamped_run_debug):
        env_hidd_states = []
        for clamped_dist in clamped_debug_list:
            max_count_hidd_state = bm.clamped_run_eval(clamped_dist, print_eval=False)
            env_hidd_states.append(stringify_vec(max_count_hidd_state))

        uniqs_len_list.append(len(np.unique(env_hidd_states)))

    return uniqs_len_list

In [3]:
bm_list = []

noise_multiplicity = 10
learning_cycles = 1000

for i in range(10):
    bm = BoltzmannMachine(env_states, 
                        num_hnodes,
                        weight_mask=weight_mask, 
                        noisy_clamping=True,
                        logging=False,
                        num_noisyEnvState_per_envState=noise_multiplicity)
    bm.learn(learning_cycles)

    last_clamped_debug_list = bm.clamped_run_debug[-1]
    last_free_run_dist = bm.free_run_debug[-1]


    print(f'#{i+1}', end=' ')
    for clamped_run_dist in last_clamped_debug_list:
        bm.clamped_run_eval(clamped_run_dist)
    bm.free_run_eval(last_free_run_dist)

    bm_list.append(bm)

  0%|          | 0/1000 [00:00<?, ?it/s]

#1 [0. 0.], [0. 1.], [1. 0.], [1. 1.], [0, 1, 2, 3, 113]


  0%|          | 0/1000 [00:00<?, ?it/s]

#2 [1. 0.], [1. 1.], [0. 1.], [0. 0.], [0, 1, 2, 3, 76, 77]


  0%|          | 0/1000 [00:00<?, ?it/s]

...

#8 [0. 1.], [1. 0.], [1. 1.], [0. 0.], [0, 1, 2, 3, 73, 88, 117, 137]


  0%|          | 0/1000 [00:00<?, ?it/s]

#9 [1. 1.], [1. 0.], [0. 1.], [0. 0.], [0, 1, 2, 3, 100, 109, 116]


  0%|          | 0/1000 [00:00<?, ?it/s]

#10 [1. 1.], [0. 1.], [0. 0.], [1. 0.], [0, 1, 2, 3, 99, 103, 143, 151]


In [4]:
for i, bm in enumerate(bm_list):
    scores = get_encNetwork_scores(bm)

    print(f'#{i+1}', end=' ')
    print(scores[-5:], end='; ')

    bm.free_run_eval(bm.free_run_debug[-1])

#1 [4, 4, 4, 4, 4]; [0, 1, 2, 3, 113]
#2 [4, 4, 4, 4, 4]; [0, 1, 2, 3, 76, 77]
#3 [4, 4, 4, 4, 4]; [0, 1, 2, 3, 93, 95, 129, 136]
#4 [4, 4, 4, 4, 4]; [0, 1, 2, 3, 79, 111, 115]
#5 [4, 4, 4, 4, 4]; [0, 1, 2, 3, 106, 110]
#6 [3, 3, 3, 3, 3]; [0, 1, 2, 5, 27, 36, 66]
#7 [4, 4, 4, 4, 4]; [0, 1, 2, 3, 73, 115, 120]
#8 [4, 4, 4, 4, 4]; [0, 1, 2, 3, 73, 88, 117, 137]
#9 [4, 4, 4, 4, 4]; [0, 1, 2, 3, 100, 109, 116]
#10 [4, 4, 4, 4, 4]; [0, 1, 2, 3, 99, 103, 143, 151]


#### Result
- First run - 4/10 
- 3/10
- 3/10
- 4/10; lower noise multiplocity of 100
- 3/10;  lower noise multiplocity of 50
- 3/10;  lower noise multiplocity of 50
- 4/10; noisy_multiplicty=30, learning_cycles=50*2
- 2/10; noisy_multiplicty=30, learning_cycles=50*2
- 5/5; noisy_multiplicty=20, learning_cycles=500*2 
- 5/5; noisy_multiplicty=20, learning_cycles=500*2 
- 6/10; noisy_multiplicty=10, learning_cycles=1000
- 7/10; noisy_multiplicty=10, learning_cycles=1000
- 7/10; noisy_multiplicty=""1"", learning_cycles=2000
- 9/10; noisy_multiplicty=10, learning_cycles=1000
- 9/10; noisy_multiplicty=10, learning_cycles=1000